# Gemini ve LangChain ile LLM API'larını Çağırma Giriş 🦜🔗

Bu notebook'ta LangChain aracılığıyla LLM API'larını nasıl kullanacağınızı öğreneceksiniz. Örnek olarak Google'ın Gemini API'sını kullanacağız. Bu notebook'un sonunda, LangChain kullanarak API çağrıları yapmayı ve bunu neden yaptığımızı bileceksiniz.

## ⚙️ Kurulum

👉 Kurulum aşamasında oluşturduğumuz `.env` dosyasındaki ortam değişkenlerini yüklemek için aşağıdaki hücreyi çalıştırın:

In [ ]:
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file

👉 Hücrenin çıktısı "`True`" mu? Harika! Artık Gemini API ile kimlik doğrulaması yapmak için kullanılacak bir `GOOGLE_API_KEY` ortam değişkeni kurmuş olduk.

Eğer değilse, yardım isteyin.

## Basit Bir API Çağrısı Yapma

Bu notebook'ta şunların nasıl yapılacağını göstereceğiz:
1. Google'ın kendi kütüphanesini kullanarak API çağrısı yapma.
2. Aynı işlemi LangChain kullanarak yapma.

## Google Generative AI Kütüphanesini Kullanma

In [ ]:
from google import genai

In [ ]:
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="What is the capital of France?",
)

`response` nesnesine bir göz atalım.

In [ ]:
response.candidates[0].content.parts[0].text

Gerçek cevabı nasıl alabileceğinizi görüyor musunuz?

Neyse ki, cevabı hemen almak için sadece `.text` özelliğini kullanabiliriz. Deneyin.

In [ ]:
response.text

Gemini cevaplarını Markdown formatında döndürür. Bunu kullanalım!

In [ ]:
from IPython.display import Markdown
Markdown(response.text)

Oluşturma parametrelerini de değiştirebilirsiniz. `google.genai` kullanarak bunu şu şekilde yaparsınız:

In [ ]:
from google import genai
from google.genai import types # We need to import types for the config

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="Write a social media post about how much you're learning about transformers.",
    config=types.GenerateContentConfig(
        max_output_tokens=200,
        temperature=1.0
    )
)

In [ ]:
Markdown(response.text)

Harika. Ancak başka bir API denemek istediğinizi düşünün, örneğin OpenAI'nin veya Anthropic'in?

Onların dokümantasyonlarını incelemek ve tüm kodunuzu onların API'sini kullanacak şekilde yeniden yazmak zorunda kalırsınız. Tabii ki benzer olacaktır, ancak aynı olmayacaktır.

Neyse ki LangChain var!

## LangChain Kullanma 🦜🔗

Neden LangChain kullanırsınız?

1. **Model-Bağımsız Kod**

   LangChain, farklı LLM sağlayıcıları (Google, OpenAI, Anthropic, vb.) arasında minimal kod değişikliği ile geçiş yapmanızı sağlayan soyutlamalar sunar. Google API'sine doğrudan kod yazarsanız, sağlayıcı değiştirmek önemli ölçüde yeniden düzenleme gerektirir.

2. **Birleşik Arayüz**

   LangChain, altta yatan API'den bağımsız olarak farklı LLM sağlayıcıları arasında etkileşimleri standartlaştırır ve tutarlı yöntemler ile yanıt formatları sunar.

3. **Bileşenlerle Çalışabilirlik**

   LangChain'in zincir ve pipeline mimarisi, tüm alt yapıyı kendiniz halletmeden prompt, bellek ve erişim sistemlerini birleştiren karmaşık iş akışları oluşturmayı kolaylaştırır.

4. **Yerleşik Araçlar**

   LangChain, çıktı ayrıştırma, prompt şablonları ve kendiniz uygulamanız gereken diğer yardımcı araçları içerir.

[LangChain'in chat entegrasyonları listesi](https://docs.langchain.com/oss/python/integrations/chat)'ne gidin ve entegrasyon listesine bakın. Favori LLM sağlayıcınızı bulabiliyor musunuz?

Kodumuzda `chat_models.ChatGoogleGenerativeAI` kullanmak istemiyoruz çünkü bu özellikle Gemini için yapılmış. LLM'yi değiştirmek istersek, modeli başlatma şeklimizi değiştirmek zorunda kalırız. Neyse ki LangChain bir modeli başlatmak için daha genel bir yol sunar.

Gemini'yi tekrar kullanalım, ancak şimdi LangChain'in genel Chat Models'ini kullanarak.

👉 [LangChain'in "Models" dokümantasyonu](https://docs.langchain.com/oss/python/langchain/models) sayfasına gidin ve Gemini kullanarak bir chat modelinin nasıl başlatılacağını bulun.

İpuçları:
1. Hemen "Basic Usage" bölümüne gidin.
2. Kullanmak istediğiniz modeli seçerek doğru dokümantasyonu hemen görebilirsiniz.

In [ ]:
# YOUR CODE HERE

Modelin en temel kullanımı sadece `.invoke()` metodunu kullanmaktır:

In [ ]:
# YOUR CODE HERE

Yanıta bir göz atalım. Nesnenin tüm öznitelik ve metodlarını içeren `__dict__`'ini güzel şekilde yazdırmak için `pprint()` kullanıyoruz.

In [ ]:
from pprint import pprint
pprint(response.__dict__)

Cevabı çıkarın ve görüntüleyin. Markdown formatında olduğunu unutmayın, bu yüzden güzel görünmesini sağlayabilirsiniz.

In [ ]:
# YOUR CODE HERE

Modelin temperature değerini `.temperature` özniteliğine erişerek kontrol edebilirsiniz. Deneyin:

In [ ]:
# YOUR CODE HERE

Modeli kullanmadan önce, özniteliklere yeni değerler atayarak oluşturma parametrelerini de ayarlayabiliriz.

Daha önce Google'ın kütüphanesini kullanarak sosyal medya gönderisi yazmak için yaptığımızın eşdeğerini kodlamaya çalışın.

> _Not_: Normal olarak modelin `max_output_tokens` değerini ayarlayabilmemiz gerekir (modeli başlatırken veya daha sonra özniteliği değiştirerek). _langchain_google_genai_'nin mevcut sürümü (4.1.1) bir [hataya](https://github.com/langchain-ai/langchain-google/issues/1454) sahip ve bu çalışmıyor. Geçici çözüm? `max_output_tokens`'ı `.invoke()` metodunun bir parametresi olarak ayarlayın.

In [ ]:
# Set the maximum number of output tokens to 200

# YOUR CODE HERE

# Set the temperature to 1.0

# YOUR CODE HERE

# Generate a response with the new settings

# YOUR CODE HERE

# Display the response

# YOUR CODE HERE


Bunun avantajı? Bu LangChain Chat Model birçok başka API'yi destekleyebilir.

Başka bir modele geçmek için değiştirmeniz gereken tek şeyler:
1. Diğer model için bir API anahtarı alın ve kodunuzda tanımlayın.
2. Modeli başlatırken model ve sağlayıcıyı değiştirin.

### Çoklu Mesajlar

`.invoke()` fonksiyonunu sadece tek bir mesajla kullanmak biraz kısıtlayıcı.

Şu gibi birden fazla mesaj sağlayabilirsiniz:
- `SystemMessage` veya sistem mesajları: modelin nasıl davranacağını söylemek için
- `HumanMessage` veya Kullanıcı mesajları: kullanıcıdan gelen girdi
- `AIMessage` veya Asistan mesajları: modelden gelen yanıt

Bir sosyal medya yazarı yapalım.

Modele nasıl davranacağını açıklayan bir sistem mesajı göndereceğiz. Sonra kullanıcı mesajında, kendimizi sadece yazacağı konuyu vermekle sınırlayabiliriz.

Bunu nasıl yapacağınızı öğrenmek için [LangChain'in "Messages" dokümantasyonu](https://docs.langchain.com/oss/python/langchain/messages)'na bakın.

Sistem mesajı için ilhama mı ihtiyacınız var? İşte başlamanız için temel bir talimat:

```python
"""Sen Üretken AI öğrencisi için gönderiler yazan yaratıcı bir sosyal medya yazarısın.
Gönderilerinde her zaman kelime oyunu ve harekete geçirici çağrı bulunur.
Gönderilerin maksimum 200 karakter uzunluğundadır.
Her zaman emoji kullanırsın.
"""
```

In [ ]:
# Import the necessary classes

# YOUR CODE HERE

# Create a list of messages

# YOUR CODE HERE

# Generate a response using the list of messages

# YOUR CODE HERE

# Display the response

# YOUR CODE HERE


🏁 Tebrikler! Artık LangChain kullanarak çoklu mesajlarla temel prompt yazma konusunda uzmanlaştınız.